# Build Chatbot

- https://python.langchain.com/docs/tutorials/chatbot/

In [23]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [24]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi!, I am bob'),
    AIMessage(content='Hello bob. How can I help you?'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)
res = llm.invoke(messages)

res.pretty_print()

================================== Ai Message ==================================

Hi, Bob!


In [38]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builider
builder = StateGraph(state_schema=MessagesState)

# Node (Graph안의 구조)
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node끼리 연결하는 선)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [39]:
# 설정(conf, config, configuration -> 설정)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방 아이디 (바뀌면 다른 대화가 된다.)

graph.invoke({'messages': messages}, config=config)

{'messages': [HumanMessage(content='Hi!, I am bob', additional_kwargs={}, response_metadata={}, id='61e7caa0-f36d-4b3c-bd25-d23452f34aa9'),
  AIMessage(content='Hello bob. How can I help you?', additional_kwargs={}, response_metadata={}, id='0498c206-3f4e-4eae-a910-469ad6587344'),
  HumanMessage(content='Say my name.', additional_kwargs={}, response_metadata={}, id='d1955abc-da13-41bd-b684-038449468c37'),
  AIMessage(content='Hi, Bob!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 33, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDL33liKgbeIiMeyYnF72VNLxXXSH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1f57ca28

In [40]:
import uuid

u_id = uuid.uuid1()
print(u_id)

config = {'configurable': {'thread_id': '가나다123'}}  # 채팅방 아이디 -> 추후에는 UUID 형식으로 생성
messages = [
    HumanMessage(content='say my name.')
]
graph.invoke({'messages': messages}, config=config)

da0d0ec3-8c53-11f0-bdbc-1063c8e7cdb0


{'messages': [HumanMessage(content='say my name.', additional_kwargs={}, response_metadata={}, id='ae11cff6-cf8a-4b34-951b-dc63bc16a648'),
  AIMessage(content="I'm sorry, but I don't have access to your name. Could you please tell me?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CDL3GMaipjdgAbLmnQ8yLqUYWTmh1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bb909ff4-977b-4577-8061-a265666d149d-0', usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'r

## Langgraph + `PromptTemplate`

In [58]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['hi']}).messages:
    print(msg)

content='너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.' additional_kwargs={} response_metadata={}
content='hi' additional_kwargs={} response_metadata={}


## State 확장

In [59]:
# 내장된 MessagesState를 확장해서 사용
class MyState(MessagesState):
    # 상속받아서 이미 key 'messages'는 있음
    # messages: Annotated[list[AnyMessage], add_messages]
    lang: str

builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

def simple_node(state: MyState):
    # prompt 추가.
    chain = prompt_template | llm  # 체인 방식
    res = chain.invoke(state)

    return {'messages': res}

builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [ ]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='Hi. I am bob')],
    'lang': 'Spanish'
}
res = graph.invoke(state, config)

for msg in res['messages']:
    msg.pretty_print()

================================ Human Message =================================

Hi. I am bob
================================== Ai Message ==================================

¡Hola, Bob! Encantado de conocerte. ¿En qué puedo ayudarte hoy?


## 대화 기록 관리하기
- 대화 내역을 관리하지 않으면, LLM의 컨텍스트 윈도우(입력 최대치)를 넘어가버림.

In [68]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# trim - 정리하다
trimmer = trim_messages(
    strategy='last',      # 최신 메세지들을
    max_tokens=65,         # 최대 65토큰 까지만 허용,
    token_counter=llm,    # llm 모델에 맞춰서 토큰 세고
    include_system=True,  # system 프롬프트는 포함(정리X)
    allow_partial=False,  # 메세지 중간에서 자르지는 말고
    start_on='human',     # 잘린 메세지의 첫번째는 사람 메세지가 되도록
    
)

messages = [
    SystemMessage(content="you're a good assistant"),

    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)  # 시스템메시지 -> 2+2 부터 등장

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [74]:
class MyState(MessagesState):
    # messages: ~~
    lang: str


builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])


def simple_node(state: MyState):
    # 메세지 정리 -> 프롬프트 생성 -> LLM 답변
    # print('정리 전 메시지 개수: ', len(state['messages']))
    trimmed_messages = trimmer.invoke(state['messages'])
    # print('정리 후 메시지 개수: ', len(trimmed_messages))
    # print('********************************************************')
    # for m in trimmed_messages:
    #     print('\t', m.pretty_print())
    # print('********************************************************')
    
    # 체인 생성
    chain = prompt_template | llm
    
    # 정리된 메세지로 state 교체 후, 체인 실행
    state['messages'] = trimmed_messages
    res = chain.invoke(state)

    return {'messages': res}


builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [76]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='라면 추천 좀')],
    'lang': '한국어'
}

# res = graph.invoke(state, config)

# # 채팅 저장은 잘 되지만, 대화내용이 잘려서 들어가는걸 확인 가능!
# for msg in res['messages']:
#     msg.pretty_print()

for chunk, metadata  in graph.stream(state, config, stream_mode='messages'):
    print(chunk.content, end='|')


|라|면| 좋아|!| 몇| 가지| 추천|해|줄|게|:

|1|.| **|신|라|면|**| (|농|심|)| –| 매|운|맛| 좋아|하는| 사람|에게| 딱|!| 얼|큰|하고| 깊|은| 맛|이| 일|품|이|야|.
|2|.| **|진|라|면|**| (|농|심|)| –| 맵|지| 않|으|면서|도| 깔|끔|한| 국|물| 맛|이| 좋아|서| 부담| 없이| 먹|기| 좋아|.
|3|.| **|삼|양| 불|닭|볶|음|면|**| –| 매|운|맛| 강|한| 걸| 좋아|한다|면| 추천|!| 매|콤|한| 볶|음|면| 스타일|이|야|.
|4|.| **|너|구|리|**| (|농|심|)| –| 해|물| 맛|이| 나는| 국|물|과| 쫄|깃|한| 면|이| 조|화|로|워|.
|5|.| **|오|뚜|기| 진|라|면| 순|한|맛|**| –| 매|운| 걸| 못| 먹|거나| 덜| 맵|게| 먹|고| 싶|을| 때| 좋아|.

|어|떤| 스타일|의| 라|면|을| 좋아|하는|지| 알려|주|면| 더| 딱| 맞|는| 추천|도| 해|줄|게|!||